In [1]:
import mlflow
import numpy as np

from encoder import CategoricalToIntegerEncoders
from format_data import (CATEGORICAL_VARIABLES, DATE_VARIABLES, ID_VARIABLES,
                         TARGET_VARIABLE)
from spark_utils import get_spark_session

In [2]:
spark = get_spark_session()

# run transform_latest.py if this don't exist
test_data = spark.read.parquet('data_transformed/amex-default-prediction/test_data_latest')
train_data = spark.read.parquet('data_transformed/amex-default-prediction/train_data_latest')
# run format_data.py if these don't exist
train_labels = spark.read.parquet('data/amex-default-prediction/train_labels')
sample_submission = spark.read.parquet('data/amex-default-prediction/sample_submission')

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/08/02 12:58:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
%%time
encs = CategoricalToIntegerEncoders(
    columns=CATEGORICAL_VARIABLES).fit(train_data)

train_pdf = train_data.join(train_labels, on='customer_ID', how='inner')
train_pdf = encs.transform(spark=spark, df=train_pdf).toPandas()

test_pdf = encs.transform(spark=spark, df=test_data).toPandas()


non_feature_columns = [
    TARGET_VARIABLE,
    *ID_VARIABLES,
    *DATE_VARIABLES.keys(),
]
feature_columns = [
    c for c in train_pdf.columns
    if c not in non_feature_columns
]
print(f'Feature columns ({len(feature_columns)}):\n' +
      ', '.join(feature_columns))

22/08/02 12:59:03 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Feature columns (189):
P_2, D_39, B_1, B_2, R_1, S_3, D_41, B_3, D_42, D_43, D_44, B_4, D_45, B_5, R_2, D_46, D_47, D_48, D_49, B_6, B_7, B_8, D_50, D_51, B_9, R_3, D_52, P_3, B_10, D_53, S_5, B_11, S_6, D_54, R_4, S_7, B_12, S_8, D_55, D_56, B_13, R_5, D_58, S_9, B_14, D_59, D_60, D_61, B_15, S_11, D_62, D_65, B_16, B_17, B_18, B_19, B_20, S_12, R_6, S_13, B_21, D_69, B_22, D_70, D_71, D_72, S_15, B_23, D_73, P_4, D_74, D_75, D_76, B_24, R_7, D_77, B_25, B_26, D_78, D_79, R_8, R_9, S_16, D_80, R_10, R_11, B_27, D_81, D_82, S_17, R_12, B_28, R_13, D_83, R_14, R_15, D_84, R_16, B_29, S_18, D_86, D_87, R_17, R_18, D_88, B_31, S_19, R_19, B_32, S_20, R_20, R_21, B_33, D_89, R_22, R_23, D_91, D_92, D_93, D_94, R_24, R_25, D_96, S_22, S_23, S_24, S_25, S_26, D_102, D_103, D_104, D_105, D_106, D_107, B_36, B_37, R_26, R_27, D_108, D_109, D_110, D_111, B_39, D_112, B_40, S_27, D_113, D_115, D_118, D_119, D_121, D_122, D_123, D_124, D_125, D_127, D_128, D_129, B_41, B_42, D_130, D_131, D_132, 

In [4]:
X_train = train_pdf[feature_columns].reset_index(drop=True)
y_train = np.array(train_pdf[TARGET_VARIABLE])
X_test = test_pdf[feature_columns].reset_index(drop=True)
print(
    f'X_train.shape: {X_train.shape} '
    f'X_test.shape: {X_test.shape} '
    f'y_train.shape: {y_train.shape} '
    f'y_train uniques: {np.unique(y_train, return_counts=True)} '
)

X_train.shape: (458913, 189) X_test.shape: (924621, 189) y_train.shape: (458913,) y_train uniques: (array([0., 1.], dtype=float32), array([340085, 118828])) 


In [5]:
from pprint import pprint
import json

experiment_id = '0'
run_id = '501e7566b38e4fafbec401ce8d8b71f3'

client = mlflow.tracking.MlflowClient()
best_run = client.get_run(run_id=run_id)
best_params = json.loads(best_run.data.params['lgb_params_json'])

# we need floats as keys here, but it's string
if 'class_weight' in best_params:
    best_params['class_weight'] = {float(k): v for k, v in best_params['class_weight'].items()}

pprint(best_params)

{'class_weight': {0.0: 1.0, 1.0: 1.4540438680770216},
 'num_iterations': 8174,
 'num_leaves': 36,
 'subsample': 0.7748260030456621}


In [6]:
%%time
from hp import get_cv_hp_metrics
import mlflow
from sklearn.model_selection import train_test_split
from pprint import pprint

X_fit, X_valid, y_fit, y_valid = train_test_split(X_train, y_train, test_size=.1)
model, metrics = get_cv_hp_metrics(
    X_train=X_fit,
    y_train=y_fit,
    X_test=X_valid,
    y_test=y_valid,
    categorical_feature=encs.columns_encoded,
    lgb_params=best_params,
    nested=False,
)
pprint(metrics)


/usr/local/lib/python3.8/dist-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/usr/local/lib/python3.8/dist-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/usr/local/lib/python3.8/dist-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['B_30_CategoricalToIntegerEncoder', 'B_38_CategoricalToIntegerEncoder', 'D_114_CategoricalToIntegerEncoder', 'D_116_CategoricalToIntegerEncoder', 'D_117_CategoricalToIntegerEncoder', 'D_120_CategoricalToIntegerEncoder', 'D_126_CategoricalToIntegerEncoder', 'D_63_CategoricalToIntegerEncoder', 'D_64_CategoricalToIntegerEncoder', 'D_66_CategoricalToIntegerEncoder', 'D_68_CategoricalToIntegerEncoder']
  _log_warning('categorical_feature in Dataset is overridde

Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[135]	valid10pct's binary_logloss: 0.232565


2022/08/02 12:59:46 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/usr/local/lib/python3.8/dist-packages/mlflow/models/signature.py:129: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."


{'test_feval_amex': 0.785574446387251,
 'test_feval_amex_gini': 0.9184643290161129,
 'test_feval_amex_top4': 0.6526845637583892}
CPU times: user 2min 5s, sys: 1.53 s, total: 2min 6s
Wall time: 23.3 s


In [7]:
from format_data import PREDICTION_VARIABLE
import pandas as pd

pred_test = pd.DataFrame({
    'customer_ID': test_pdf['customer_ID'],
    PREDICTION_VARIABLE: model.predict(X_test, raw_score=True),
})
pred_test.to_csv('submission.csv', index=False)
pred_test.head()

,customer_ID,prediction
0,61fb2dd47f7a2eb2e81c3bf89e147bc61d6a42914cf570...,-5.342020
1,117a23d25e3b45d80250045da0d9a1bd50a7f57468cf5e...,0.987675
2,344029ccdb720001501d138a9a5ab1ff7abf827d296bba...,0.618245
3,4b09ee54b4254d63fa938bb67b93b9c134f336d78945d5...,-1.551953
4,c4e6c35ecccc7d71d7d677513fe15cb52dd034420cad2b...,-6.147565
